In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class DiffAttention(nn.Module):
    def __init__(self, embed_size, heads):
        super(DiffAttention, self).__init__()
        self.embed_size = embed_size
        self.heads = heads
        self.head_dim = embed_size // heads

        assert (
            self.head_dim * heads == embed_size
        ), "Embedding size must be divisible by heads"

        self.values = nn.Linear(embed_size, embed_size, bias=False)
        self.keys = nn.Linear(embed_size, embed_size, bias=False)
        self.queries = nn.Linear(embed_size, embed_size, bias=False)
        self.fc_out = nn.Linear(embed_size, embed_size)

    def forward(self, x):
        N, seq_length, _ = x.shape
        values = self.values(x)
        keys = self.keys(x)
        queries = self.queries(x)

        # Split into heads
        values = values.view(N, seq_length, self.heads, self.head_dim)
        keys = keys.view(N, seq_length, self.heads, self.head_dim)
        queries = queries.view(N, seq_length, self.heads, self.head_dim)

        values = values.permute(0, 2, 1, 3)  # (N, heads, seq_length, head_dim)
        keys = keys.permute(0, 2, 1, 3)      # (N, heads, seq_length, head_dim)
        queries = queries.permute(0, 2, 1, 3)  # (N, heads, seq_length, head_dim)

        # Calculate attention scores
        energy = torch.einsum("nqhd,nkhd->nhqk", [queries, keys])  # (N, heads, seq_length, seq_length)
        attention = F.softmax(energy, dim=3)

        # Apply attention to values
        out = torch.einsum("nhql,nlhd->nqhd", [attention, values]).reshape(N, seq_length, self.heads * self.head_dim)
        out = self.fc_out(out)

        return out

class DiffTransformer(nn.Module):
    def __init__(self, embed_size, heads, num_layers, input_dim):
        super(DiffTransformer, self).__init__()
        self.embed_size = embed_size
        self.num_layers = num_layers
        self.attention = DiffAttention(embed_size, heads)
        self.fc = nn.Linear(input_dim, embed_size)

    def forward(self, x):
        x = self.fc(x)  # Project input to embedding size
        for _ in range(self.num_layers):
            x = self.attention(x)
        return x

# Example usage
if __name__ == "__main__":
    embed_size = 256  # Embedding size
    heads = 8         # Number of attention heads
    num_layers = 6    # Number of transformer layers
    input_dim = 128   # Input feature dimension
    seq_length = 50   # Length of input sequences
    batch_size = 32   # Batch size

    model = DiffTransformer(embed_size, heads, num_layers, input_dim)
    x = torch.rand(batch_size, seq_length, input_dim)  # Random input
    output = model(x)
    print(output.shape)  # Should output (batch_size, seq_length, embed_size)



torch.Size([32, 50, 256])


# Feature Ideas

### Economic
* days from an election
* Fed Funds rate at beginning of the month
* Previous year's US GDP
* Previous year's Eurozone GDP (From world bank)
* Price of otr 10y if possible

### Technical
* long n day vol
* short n day vol
* long/short n day moving average
* If reliable volume can be acquired -> long/short vwap

In [139]:
import pandas as pd, numpy as np, pandas_datareader.data as web, pandas_datareader.wb as wb, requests, os

series_id = 'FEDFUNDS'
start_date = '01-01-2000'
end_date = '11-01-2024'
long_window = 21
short_window = 7


fedFunds = web.DataReader(series_id, 'fred', start_date, end_date)
fedFunds['monthYear'] = [f'{time.month}-{time.year}' for time in fedFunds.index]
fedFunds

,FEDFUNDS,monthYear
DATE,,
2000-01-01,5.45,1-2000
2000-02-01,5.73,2-2000
2000-03-01,5.85,3-2000
2000-04-01,6.02,4-2000
2000-05-01,6.27,5-2000
...,...,...
2024-05-01,5.33,5-2024
2024-06-01,5.33,6-2024
2024-07-01,5.33,7-2024


In [117]:
euroGDP = wb.WorldBankReader(countries='EMU', symbols='NY.GDP.MKTP.CD', start=start_date)
euroGDP = euroGDP.read()['NY.GDP.MKTP.CD'].reset_index([0,1])
euroGDP.columns = ['country', 'year','euroGDP']
euroGDP

d:\Program Files (x86)\python\Lib\site-packages\pandas_datareader\wb.py:592: UserWarning: Non-standard ISO country codes: EMU
  warnings.warn(
C:\Users\nicol\AppData\Local\Temp\ipykernel_9692\4221959429.py:2: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  euroGDP = euroGDP.read()['NY.GDP.MKTP.CD'].reset_index([0,1])


,country,year,euroGDP
0,Euro area,2023,1.554486e+13
1,Euro area,2022,1.422435e+13
2,Euro area,2021,1.475463e+13
3,Euro area,2020,1.315516e+13
4,Euro area,2019,1.348115e+13
5,Euro area,2018,1.376112e+13
6,Euro area,2017,1.273690e+13
7,Euro area,2016,1.202590e+13
8,Euro area,2015,1.172711e+13
9,Euro area,2014,1.357007e+13


In [100]:
response = requests.get(f'https://api.polygon.io/v2/aggs/ticker/C:EURUSD/range/1/day/2000-01-01/2024-10-28?adjusted=true&sort=asc&apiKey={os.environ["polygonKey"]}').json()

,volume,vwap,open,close,high,low,time,transactions,yearMonth,year
0,6749,0.9955,0.99348,0.99543,0.99619,0.99300,2022-10-30,6749,10-2022,2022
1,201077,0.9917,0.99542,0.98856,0.99656,0.98720,2022-10-31,201077,10-2022,2022
2,228463,0.9904,0.98855,0.98771,0.99539,0.98520,2022-11-01,228463,11-2022,2022
3,244289,0.9880,0.98750,0.98133,0.99754,0.98047,2022-11-02,244289,11-2022,2022
4,228775,0.9776,0.98133,0.97492,0.98394,0.97290,2022-11-03,228775,11-2022,2022
...,...,...,...,...,...,...,...,...,...,...
651,168780,1.0783,1.07961,1.07804,1.08070,1.07600,2024-10-23,168780,10-2024,2024
652,152416,1.0802,1.07804,1.08235,1.08310,1.07700,2024-10-24,152416,10-2024,2024
653,153720,1.0819,1.08239,1.07950,1.08400,1.07900,2024-10-25,153720,10-2024,2024
654,2961,1.0795,1.07929,1.07959,1.07999,1.07850,2024-10-27,2961,10-2024,2024


In [143]:
eurusd = pd.DataFrame(response['results'])
eurusd.t = pd.to_datetime(eurusd.t, unit='ms')
eurusd.columns = [
    'volume',
    'vwap',
    'open',
    'close',
    'high',
    'low',
    'time',
    'transactions'
]
eurusd['monthYear'] = [f'{time.month}-{time.year}' for time in eurusd.time]
eurusd['prevYear'] = [str(int(time.year) - 1) for time in eurusd.time]
eurusd['euroGDP'] = [euroGDP[euroGDP.year == year].euroGDP.values[0] for year in eurusd.prevYear]
eurusd = pd.merge(eurusd, fedFunds, how='left', on='monthYear')
eurusd = eurusd[~pd.isna(eurusd.FEDFUNDS)]
eurusd

,volume,vwap,open,close,high,low,time,transactions,monthYear,prevYear,euroGDP,FEDFUNDS
0,6749,0.9955,0.99348,0.99543,0.99619,0.99300,2022-10-30,6749,10-2022,2021,1.475463e+13,3.08
1,201077,0.9917,0.99542,0.98856,0.99656,0.98720,2022-10-31,201077,10-2022,2021,1.475463e+13,3.08
2,228463,0.9904,0.98855,0.98771,0.99539,0.98520,2022-11-01,228463,11-2022,2021,1.475463e+13,3.78
3,244289,0.9880,0.98750,0.98133,0.99754,0.98047,2022-11-02,244289,11-2022,2021,1.475463e+13,3.78
4,228775,0.9776,0.98133,0.97492,0.98394,0.97290,2022-11-03,228775,11-2022,2021,1.475463e+13,3.78
...,...,...,...,...,...,...,...,...,...,...,...,...
627,148887,1.1180,1.11903,1.11296,1.12140,1.11200,2024-09-25,148887,9-2024,2023,1.554486e+13,5.13
628,136010,1.1160,1.11297,1.11768,1.11893,1.11250,2024-09-26,136010,9-2024,2023,1.554486e+13,5.13
629,182226,1.1163,1.11769,1.11640,1.12031,1.11240,2024-09-27,182226,9-2024,2023,1.554486e+13,5.13
630,2926,1.1168,1.11527,1.11680,1.11721,1.11527,2024-09-29,2926,9-2024,2023,1.554486e+13,5.13


In [155]:
features = eurusd[['volume', 'vwap', 'open', 'close', 'high', 'low', 'euroGDP', 'FEDFUNDS']]

features['long_vwap'] = [np.mean(features.vwap[i - long_window:i]) if i - long_window > 0 else None for i in range(len(features))]
features['short_vwap'] = [np.mean(features.vwap[i - short_window:i]) if i - short_window > 0 else None for i in range(len(features))]

#add other features, standardize, then modify the difftransformer's output to 1 node

features

,volume,vwap,open,close,high,low,euroGDP,FEDFUNDS
0,6749,0.9955,0.99348,0.99543,0.99619,0.99300,1.475463e+13,3.08
1,201077,0.9917,0.99542,0.98856,0.99656,0.98720,1.475463e+13,3.08
2,228463,0.9904,0.98855,0.98771,0.99539,0.98520,1.475463e+13,3.78
3,244289,0.9880,0.98750,0.98133,0.99754,0.98047,1.475463e+13,3.78
4,228775,0.9776,0.98133,0.97492,0.98394,0.97290,1.475463e+13,3.78
...,...,...,...,...,...,...,...,...
627,148887,1.1180,1.11903,1.11296,1.12140,1.11200,1.554486e+13,5.13
628,136010,1.1160,1.11297,1.11768,1.11893,1.11250,1.554486e+13,5.13
629,182226,1.1163,1.11769,1.11640,1.12031,1.11240,1.554486e+13,5.13
630,2926,1.1168,1.11527,1.11680,1.11721,1.11527,1.554486e+13,5.13
